# Phase 2: National Data Merger
**Project:** National Rent Intelligence Engine  
**Goal:** Clean, impute, and merge disparate data sources (CMHC, CREA, IRCC) into a single master training dataset.  
**Date:** February 2026

In [11]:
import pandas as pd
import numpy as np
import os
import glob

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# ==========================================
# 1. DEFINE PATHS (UPDATED WITH AUTO-DETECT)
# ==========================================
if os.path.exists("data/raw"):
    RAW_PATH = "data/raw"
    PROCESSED_PATH = "data/processed"
elif os.path.exists("../data/raw"):
    RAW_PATH = "../data/raw"
    PROCESSED_PATH = "../data/processed"
else:
    # Fallback default
    RAW_PATH = "../data/raw"
    PROCESSED_PATH = "../data/processed"
    print("⚠️ Warning: Could not auto-detect data paths. Defaulting to '../data/raw'")

os.makedirs(PROCESSED_PATH, exist_ok=True)

print(f"Current Working Directory: {os.getcwd()}")
print(f"Using Raw Data Path:       {RAW_PATH}")

# ==========================================
# 2. DEFINE CITY MAPPING
# ==========================================
# We need to standardize City Names across 3 different agencies
city_map = {
    # CREA Name : Standard Name
    'GREATER_TORONTO': 'Toronto',
    'GREATER_VANCOUVER': 'Vancouver',
    'CALGARY': 'Calgary',
    'EDMONTON': 'Edmonton',
    'OTTAWA': 'Ottawa',
    'MONTREAL_CMA': 'Montreal',
    'HALIFAX_DARTMOUTH': 'Halifax',
    'WINNIPEG': 'Winnipeg',
    'VICTORIA': 'Victoria',
    'HAMILTON_BURLINGTON': 'Hamilton',
    'KITCHENER_WATERLOO': 'Kitchener',
    'LONDON_ST_THOMAS': 'London',
    'WINDSOR_ESSEX': 'Windsor',
    'QUEBEC_CMA': 'Quebec City',
    'ST_JOHNS_NL': 'St. Johns',
    'SASKATOON': 'Saskatoon',
    'REGINA': 'Regina'
}

# 3. DEFINE PROVINCE MAPPING (For Student Visa Broadcast)
prov_map = {
    'Toronto': 'Ontario',
    'Ottawa': 'Ontario',
    'Hamilton': 'Ontario',
    'Kitchener': 'Ontario',
    'London': 'Ontario',
    'Windsor': 'Ontario',
    'Vancouver': 'British Columbia',
    'Victoria': 'British Columbia',
    'Calgary': 'Alberta',
    'Edmonton': 'Alberta',
    'Montreal': 'Quebec',
    'Quebec City': 'Quebec',
    'Halifax': 'Nova Scotia',
    'Winnipeg': 'Manitoba',
    'Saskatoon': 'Saskatchewan',
    'Regina': 'Saskatchewan',
    'St. Johns': 'Newfoundland and Labrador'
}

print("Configuration loaded. Mappings defined.")

Current Working Directory: /Users/abigail/Downloads/DS Projects/RentPulse Ontario/National_Insight_Engine/src
Using Raw Data Path:       ../data/raw
Configuration loaded. Mappings defined.


## 1. Process CMHC Data (Supply & Turnover)
We loop through the years (2019-2025) and extract:
* **Turnover Rate** from `Table 1.0`
* **Rental Universe (Total Units)** from `Table 4.1`

In [14]:
def clean_cmhc_turnover_excel(file_path, year):
    """Extracts Turnover Rate from Table 1.0 inside the Excel file"""
    try:
        # Load raw data to find the header
        df_raw = pd.read_excel(file_path, sheet_name="Table 1.0", header=None)
        
        # STRICTER SEARCH: Find row where a cell is EXACTLY "Centre"
        # This prevents matching "Major Centres" in the title row
        header_row = df_raw[df_raw.astype(str).apply(lambda x: x.str.strip().eq('Centre').any(), axis=1)].index[0]
        
        # Reload with correct header
        df = pd.read_excel(file_path, sheet_name="Table 1.0", header=header_row)
        
        # Rename first column to 'Centre' to be safe (handles whitespace issues)
        df.rename(columns={df.columns[0]: 'Centre'}, inplace=True)
        
        # Filter for valid rows
        df = df.dropna(subset=['Centre'])
        
        # Standardize City Names
        df['Standard_City'] = df['Centre'].astype(str).str.replace(' CMA', '').str.replace(' CA', '').str.strip()
        
        # Find Turnover Columns (Look for "Turnover" or "%")
        # In this specific dataset, we want the LAST column with "Oct" in it, 
        # which usually corresponds to the most recent Turnover Rate.
        oct_cols = [c for c in df.columns if "Oct" in str(c)]
        
        # Logic: Vacancy (2 cols) -> Turnover (2 cols). We want the 4th Oct column.
        if len(oct_cols) >= 4:
            target_col = oct_cols[3] 
        else:
            target_col = oct_cols[-1] # Fallback

        df_out = df[['Standard_City', target_col]].copy()
        df_out.columns = ['City', 'Turnover_Rate']
        df_out['Year'] = year
        
        # Clean numeric
        df_out['Turnover_Rate'] = pd.to_numeric(df_out['Turnover_Rate'], errors='coerce')
        
        return df_out
    except Exception as e:
        print(f"Error reading Turnover in {os.path.basename(file_path)}: {e}")
        return pd.DataFrame()

def clean_cmhc_universe_excel(file_path, year):
    """Extracts Universe from Table 4.1 inside the Excel file"""
    try:
        # Load Table 4.1
        df_raw = pd.read_excel(file_path, sheet_name="Table 4.1", header=None)
        
        # STRICTER SEARCH for header
        header_row = df_raw[df_raw.astype(str).apply(lambda x: x.str.strip().eq('Centre').any(), axis=1)].index[0]
        
        df = pd.read_excel(file_path, sheet_name="Table 4.1", header=header_row)
        df.rename(columns={df.columns[0]: 'Centre'}, inplace=True)
        
        # Clean City
        df['Standard_City'] = df['Centre'].astype(str).str.replace(' CMA', '').str.replace(' CA', '').str.strip()
        
        # Find Universe Column (Look for 'Universe' or 'Apts in the RMS')
        univ_col = [c for c in df.columns if "Universe" in str(c) or "Apts" in str(c)][0]
        
        df_out = df[['Standard_City', univ_col]].copy()
        df_out.columns = ['City', 'Total_Units']
        df_out['Year'] = year
        
        # Clean numeric
        df_out['Total_Units'] = pd.to_numeric(df_out['Total_Units'].astype(str).str.replace(',', ''), errors='coerce')
        
        return df_out
    except Exception as e:
        print(f"Error reading Universe in {os.path.basename(file_path)}: {e}")
        return pd.DataFrame()

print("✅ Parsers Updated (Stricter Header Logic)")

✅ Parsers Updated (Stricter Header Logic)


In [15]:
years = range(2019, 2026) # 2019 to 2025
cmhc_data = []

print(f"Starting merge process using path: {RAW_PATH} ...")

for y in years:
    file_name = f"Turnover-Rates-{y}.xlsx"
    file_path = os.path.join(RAW_PATH, file_name)
    
    if os.path.exists(file_path):
        print(f"Processing {y}...")
        
        try:
            # Extract
            df_turn = clean_cmhc_turnover_excel(file_path, y)
            df_univ = clean_cmhc_universe_excel(file_path, y)
            
            # Merge if both exist
            if not df_turn.empty and not df_univ.empty:
                df_merged = pd.merge(df_turn, df_univ, on=['City', 'Year'], how='inner')
                cmhc_data.append(df_merged)
            else:
                print(f"  -> ⚠️ Data empty for {y}. Turnover Rows: {len(df_turn)}, Universe Rows: {len(df_univ)}")
                
        except Exception as e:
            print(f"  -> ❌ Unexpected error for {y}: {e}")
            
    else:
        print(f"  -> Skipping {y} (File not found)")

if cmhc_data:
    df_cmhc = pd.concat(cmhc_data)
    
    # Manual City Name Fixes
    city_corrections = {
        'St. John\'s': 'St. Johns', 
        'Montréal': 'Montreal', 
        'Québec': 'Quebec City'
    }
    df_cmhc['City'] = df_cmhc['City'].replace(city_corrections)
    
    print(f"\n✅ SUCCESS: Total CMHC Rows: {len(df_cmhc)}")
    display(df_cmhc.head())
else:
    print("\n❌ ERROR: No data was merged.")

Starting merge process using path: ../data/raw ...
Processing 2019...
Processing 2020...
Processing 2021...
Processing 2022...
Processing 2023...
Processing 2024...
Processing 2025...

✅ SUCCESS: Total CMHC Rows: 161


,City,Turnover_Rate,Year,Total_Units
0,Halifax,18.8,2019,3.8
1,Montreal,15.7,2019,1.8
2,Ottawa-Gatineau (Qué. part),21.3,2019,1.6
3,Quebec City,21.5,2019,1.4
4,Hamilton,14.0,2019,0.2


Data audit

In [16]:
# --- DATA AUDIT BLOCK ---
print(f"Total Rows Loaded: {len(df_cmhc)}")
print(f"Years Covered: {df_cmhc['Year'].unique()}")
print("\n--- Row Counts per City (Should be ~7 for 2019-2025) ---")
print(df_cmhc['City'].value_counts())

print("\n--- Sample of Toronto Data ---")
# Check if Toronto data actually has numbers
check_city = df_cmhc[df_cmhc['City'].str.contains("Toronto")]
if not check_city.empty:
    display(check_city.sort_values('Year'))
else:
    print("⚠️ WARNING: Toronto data is missing!")

print("\n--- Check for Missing Values ---")
print(df_cmhc.isnull().sum())

Total Rows Loaded: 161
Years Covered: [2019 2020 2021 2022 2023 2024 2025]

--- Row Counts per City (Should be ~7 for 2019-2025) ---
City
Halifax                                                                                                                                   7
Montreal                                                                                                                                  7
Ottawa-Gatineau (Qué. part)                                                                                                               7
Quebec City                                                                                                                               7
Hamilton                                                                                                                                  7
Kitchener-Cambridge-Waterloo                                                                                                              7
London                

,City,Turnover_Rate,Year,Total_Units
8,Toronto,9.5,2019,0.8
8,Toronto,8.0,2020,1.9
8,Toronto,14.4,2021,1.7
8,Toronto,9.8,2022,1.1
8,Toronto,8.3,2023,0.6
8,Toronto,6.4,2024,0.7
8,Toronto,8.7,2025,0.9



--- Check for Missing Values ---
City              0
Turnover_Rate    42
Year              0
Total_Units      46
dtype: int64


### 📝 Strategic Data Methodology

To ensure the "National Insight Engine" prioritizes **precision over noise**, we enforced the following two strategic data exclusions:

**1. Valuation Source: "Not Seasonally Adjusted (Monthly)" Only**
* **Decision:** We exclusively use the **Monthly, Non-Seasonally Adjusted (NSA)** price data from CREA.
* **Reason:** The CMHC Rental Survey occurs specifically in **October**. Annual averages smooth out this signal, and "Seasonally Adjusted" prices are statistical abstractions. Using raw October NSA prices ensures our **Cap Rate** calculation ($Rent \div Price$) reflects the actual market reality at the exact moment of survey.

**2. Historical Cutoff: Excluded 'Primary Engine.csv' (2015–2018)**
* **Decision:** We omitted the historical rent file to focus the model on the **2019–2025** era.
* **Reason:** The older data lacks the granular **Turnover Rate** and **Rental Universe** metrics found in modern reports. Including it would force us to impute (guess) nearly 40% of our training features. We prioritize a shorter, high-quality "Modern Era" dataset over a longer, lower-quality one to better predict post-pandemic market shifts.

In [17]:
# ---------------------------------------------------------
# 2. PROCESS CREA DATA (Handling Multi-Sheet Excel)
# ---------------------------------------------------------
crea_file_path = os.path.join(RAW_PATH, "Not Seasonally Adjusted (M).xlsx")
crea_data = []

if os.path.exists(crea_file_path):
    print(f"Found CREA Excel: {crea_file_path}")
    print("Reading sheets... (This might take a moment)")
    
    # Load the Excel file wrapper to list all sheet names
    xls = pd.ExcelFile(crea_file_path)
    print(f"Sheets found: {xls.sheet_names[:5]} ... and more")
    
    for sheet in xls.sheet_names:
        # Match Sheet Name to Our Standard City Name
        # We look for the sheet name in our city_map keys
        # E.g. Sheet "GREATER_TORONTO" -> Map to "Toronto"
        
        # Check if this sheet matches any of our target keys
        matched_city = None
        for key, std_name in city_map.items():
            if key in sheet: 
                matched_city = std_name
                break
        
        if matched_city:
            # Read that specific sheet
            df = pd.read_excel(xls, sheet_name=sheet)
            
            # Ensure Date parsing
            df['Date'] = pd.to_datetime(df['Date'])
            df['Year'] = df['Date'].dt.year
            
            # We need Apartment Benchmark. Priority: Apartment_Benchmark > Apartment_HPI
            if 'Apartment_Benchmark' in df.columns:
                # Group by Year to get Annual Average Price
                df_annual = df.groupby('Year')['Apartment_Benchmark'].mean().reset_index()
                df_annual['City'] = matched_city
                df_annual.columns = ['Year', 'Buy_Price', 'City']
                crea_data.append(df_annual)
            else:
                print(f"  -> Sheet {sheet} missing 'Apartment_Benchmark'")
                
    if crea_data:
        df_crea = pd.concat(crea_data)
        print(f"\n✅ CREA Data Loaded! Found prices for: {df_crea['City'].unique()}")
        display(df_crea.head())
    else:
        print("❌ Error: No matching cities found in CREA Excel.")

else:
    print(f"❌ Error: CREA file not found at {crea_file_path}")

Found CREA Excel: ../data/raw/Not Seasonally Adjusted (M).xlsx
Reading sheets... (This might take a moment)
Sheets found: ['AGGREGATE', 'BRITISH_COLUMBIA', 'VANCOUVER_ISLAND', 'VICTORIA', 'LOWER_MAINLAND'] ... and more

✅ CREA Data Loaded! Found prices for: <StringArray>
['Victoria', 'Vancouver', 'Calgary', 'Edmonton', 'Regina', 'Saskatoon', 'Winnipeg', 'Hamilton', 'Kitchener', 'London', 'Ottawa', 'Toronto', 'Windsor', 'Montreal', 'Quebec City', 'Halifax', 'St. Johns']
Length: 17, dtype: str


,Year,Buy_Price,City
0,2005,190116.666667,Victoria
1,2006,223050.000000,Victoria
2,2007,247725.000000,Victoria
3,2008,262925.000000,Victoria
4,2009,252033.333333,Victoria
